Connect to the server and the database.

In [1]:
import mysql.connector as connector

try:
    connection = connector.connect(user="newuser", password="user2023#Password", db = "littleLemonDB")
except connector.Error as err:
    print("Error:", err)
cursor = connection.cursor()
print("The cursor is created.")

Error: 1045 (28000): Access denied for user 'newuser'@'localhost' (using password: YES)


NameError: name 'connection' is not defined

Show all the tables.

In [ ]:
show_tables_query = "SHOW tables" 
cursor.execute(show_tables_query)
results=cursor.fetchall
print(results)

In [ ]:
filter_cost_query = """SELECT customers.customer_name, orders.order_cost
FROM customers
JOIN orders ON customers.customer_id = orders.customer_id
WHERE orders.order_cost > 50;"""
cursor.execute(filter_cost_query)
results=cursor.fetchall
print(results)

Virtual Tables to Summarize Data

In [ ]:
ordersview_query ="""CREATE VIEW ordersview AS
    SELECT
        Orders.Orders_ID,
        Orders.Quantity,
        Orders.Total Cost
    FROM
        Orders
    WHERE
        (Orders.Quantity > 2)"""
cursor.execute(ordersview_query)
results=cursor.fetchall
print(results)

In [ ]:
four_table_view_query = """CREATE VIEW four_table_view AS
SELECT
    Customers.Customer_ID,
    Customers.Name,
    Orders.Order_ID,
    Orders.Cost,
    Menus.Name,
    MenusItems.Course,
    MenusItems.Starter
FROM
    Customers
    JOIN Orders ON Customers.Name = Orders.Order_ID
    JOIN Menus ON Orders.Cost = Menus.Name
    JOIN MenuItems ON Menus.Name = MenuItems.Course
ORDER BY Orders.Cost DESC;"""
cursor.execute(four_table_view_query)
results=cursor.fetchall
print(results)

In [ ]:
more_than_two_query = """CREATE VIEW more_than_two AS
SELECT Names
FROM Menus
WHERE Menu_ID = ANY (
    SELECT Menu_ID
    FROM Orders
    WHERE quantity > 2
);"""
cursor.execute(more_than_two_query)
results=cursor.fetchall
print(results)

Get Max Quantity

In [ ]:
get_max_quantity_query ="""CREATE PROCEDURE GetMaxQuantity()
BEGIN
   DECLARE maxQuantity INT;

   SELECT MAX(quantity) INTO maxQuantity
    FROM orders;

   SELECT maxQuantity AS 'Maximum Quantity';
END //"""
call_max_query = """CALL GetMaxQuantity();"""
cursor.execute(get_max_quantity_query)
cursor.execute(call_max_query)
results=cursor.fetchall
print(results)

Get Order Detail

In [ ]:
get_order_detail_query ="""DECLARE id INT;
SET id = 1;

DECLARE Order_ID INT;
DECLARE quantity INT;
DECLARE cost DECIMAL(10, 2);

PREPARE GetOrderDetail (INT) AS
    SELECT Order_ID, quantity, cost
    FROM Orders
    WHERE Customer_ID = $1;

EXECUTE GetOrderDetail USING inputID;"""
cursor.execute(get_order_detail_query)
results=cursor.fetchall
print(results)

Cancel Order

In [ ]:
cancel_order_query = """CREATE PROCEDURE CancelOrder(IN id INT)
BEGIN
        DELETE FROM Orders WHERE Orders_ID = id;
END //"""
cursor.execute(cancel_order_query)

Bookings Procedure

In [ ]:
insert_into_bookings_query = """INSERT INTO Bookings (Booking_ID, Booking_Date, TableNumber, Customer_ID)
VALUES
    (1, 2022-10-10, 5, 1),
    (2, 2022-11-12, 3, 3),
    (3, 2022-10-11, 2, 2),
    (4, 2022-10-13, 2, 1);"""
cursor.execute(insert_into_bookings_query)

In [ ]:
check_bookings_query = """CREATE PROCEDURE CheckBooking(IN p_date DATE, IN p_table_number INT)
BEGIN
    DECLARE table_count INT;

    SELECT COUNT(*) INTO table_count
    FROM bookings
    WHERE booking_date = p_date AND table_number = p_table_number;

    IF room_count > 0 THEN
        SELECT CONCAT('Room ', p_table_number, ' is booked') AS result_message;
    ELSE
        SELECT CONCAT('Room ', p_table_number, ' is available') AS result_message;
    END IF;
END //"""
cursor.execute(check_bookings_query)

In [ ]:
add_valid_booking_query = """CREATE PROCEDURE AddValidBooking(IN p_date DATE, IN p_table_number INT, IN p_guest_name VARCHAR(255))
BEGIN
    DECLARE table_count INT;

    START TRANSACTION;

    SELECT COUNT(*) INTO table_count
    FROM bookings
    WHERE booking_date = p_date AND table_number = p_table_number;

    IF table_count = 0 THEN
        INSERT INTO bookings (booking_date, table_number, guest_name)
        VALUES (p_date, p_table_number, p_guest_name);

        COMMIT;

        SELECT CONCAT('Booking for Table ', p_table_number, ' on ', p_date, ' is successful') AS result_message;
    ELSE
        ROLLBACK;

        SELECT CONCAT('Room ', p_table_number, ' is already booked on ', p_date) AS result_message;
    END IF;
END //"""
cursor.execute(add_valid_booking_query)

In [ ]:
add_booking_query = """CREATE PROCEDURE AddBooking(
    IN p_Booking_ID INT,
    IN p_Name VARCHAR(255),
    IN p_Date DATE,
    IN p_table_Number INT
)
BEGIN
    INSERT INTO bookings (Booking_ID, Name, Date, table_Number)
    VALUES (p_Booking_ID, p_Name, p_Date, p_table_Number);
END //"""

In [ ]:
update_booking_query = """CREATE PROCEDURE UpdateBookingDate(
    IN p_Booking_ID INT,
    IN p_New_Date DATE
)
BEGIN
    UPDATE bookings
    SET Date = p_New_Date
    WHERE Booking_ID = p_Booking_ID;
END //
"""

In [ ]:
cancel_booking_query = """CREATE PROCEDURE CancelBooking(
    IN p_Booking_ID INT
)
BEGIN
    DELETE FROM bookings
    WHERE Booking_ID = p_Booking_ID;
END //"""